In [1]:
import os
os.environ["HF_HOME"] = "//mnt/data/stt/Datasets/Datasets/huggingface-cache"
os.environ["HF_DATASETS_CACHE"] = f"{os.environ['HF_HOME']}/datasets"
os.environ["TRANSFORMERS_CACHE"] = f"{os.environ['HF_HOME']}/transformers"
os.environ["HF_METRICS_CACHE"] = f"{os.environ['HF_HOME']}/metrics"
print("HF_DATASETS_CACHE:", os.environ["HF_DATASETS_CACHE"])

HF_DATASETS_CACHE: //mnt/data/stt/Datasets/Datasets/huggingface-cache/datasets


In [2]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset, Audio
import shutil
import soundfile as sf
from tqdm import tqdm
from IPython.display import Audio, display

In [3]:
os.getcwd()

'/mnt/data/stt/Datasets/Datasets'

In [4]:
from huggingface_hub import login

In [ ]:
login(token="HF-TOKEN")  # Replace with your token

In [6]:
from datasets import load_dataset
bhasaanuvaad = load_dataset("ai4bharat/Spoken-Tutorial", "indic2en", split="gujarati")


README.md:   0%|          | 0.00/6.33k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/348M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/477M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/572M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/569M [00:00<?, ?B/s]

...

Generating punjabi split:   0%|          | 0/8864 [00:00<?, ? examples/s]

Generating sanskrit split:   0%|          | 0/7800 [00:00<?, ? examples/s]

Generating nepali split:   0%|          | 0/1088 [00:00<?, ? examples/s]

Generating tamil split:   0%|          | 0/35702 [00:00<?, ? examples/s]

Generating telugu split:   0%|          | 0/6257 [00:00<?, ? examples/s]

In [7]:
print(bhasaanuvaad)

Dataset({
    features: ['chunked_audio_filepath', 'text', 'pred_text', 'audio_filepath', 'start_time', 'duration', 'alignment_score', 'video_id', 'en_mining_score', 'en_text'],
    num_rows: 34647
})


In [8]:
print(bhasaanuvaad.column_names)

['chunked_audio_filepath', 'text', 'pred_text', 'audio_filepath', 'start_time', 'duration', 'alignment_score', 'video_id', 'en_mining_score', 'en_text']


In [9]:
print(bhasaanuvaad[0])

{'chunked_audio_filepath': {'path': '0.wav', 'array': array([ 0.        ,  0.        ,  0.        , ...,  0.00121638,
       -0.00020479, -0.00078996], shape=(89600,)), 'sampling_rate': 16000}, 'text': 'નમસ્તે મિત્રો ઇન્ટ્રોડક્શન ટીઓ જીચેમ્પેઇન્ટ પરનાં આ ટ્યુટોરીયલમાં સ્વાગત છે', 'pred_text': 'નમસ્તે મિત્રો ઇન્ટ્રોડક્શન ટુ જી કેમ પેઇન્ટ પરના આ ટ્યુટોરિયલમાં સ્વાગત છે', 'audio_filepath': '/data/speech-translation/spoken_output/spoken_tutorial_output1/final_audios/gujarati/J_nmNmLVNLM.gu.wav', 'start_time': 0.96, 'duration': 5.6, 'alignment_score': 0.9530201342281879, 'video_id': 'J_nmNmLVNLM', 'en_mining_score': 0.2737593650817871, 'en_text': 'With this, we come to the end of this tutorial'}


**From extraction to saving final csv pipeline**

In [10]:
from datasets import load_dataset, Audio as DatasetsAudio
from IPython.display import Audio as DisplayAudio, display

In [11]:
class IndicVoiceProcessor:
    def __init__(self, dataset_name, language, audio_column, text_column, output_audio_dir, output_csv_path, final_audio_folder):
        self.dataset = load_dataset(dataset_name, "indic2en", split=language)
        self.dataset = self.dataset.cast_column(audio_column, DatasetsAudio())
        self.audio_column = audio_column
        self.text_column = text_column
        self.output_audio_dir = output_audio_dir
        self.output_csv_path = output_csv_path
        self.final_audio_folder = final_audio_folder
        os.makedirs(self.output_audio_dir, exist_ok=True)
        os.makedirs(self.final_audio_folder, exist_ok=True)

    def extract_and_save_audio(self):
        records = []

        if os.listdir(self.output_audio_dir): # Check and empty the folder if not empty
            print(f" Folder '{self.output_audio_dir}' is not empty. Clearing contents...")
            for file in os.listdir(self.output_audio_dir):
                file_path = os.path.join(self.output_audio_dir, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print(" Folder cleared.")

        print("Extracting and saving audio files...")
        for idx, example in tqdm(enumerate(self.dataset), total=len(self.dataset)):
            audio_data = example[self.audio_column]
            audio_array = audio_data["array"]
            sample_rate = audio_data["sampling_rate"]
            duration_sec = len(audio_array) / sample_rate
            duration_ms = duration_sec * 1000
            audio_filename = f"bashaanuvaadh_spo-tut_gu_{idx}.wav"
            audio_path = os.path.join(self.output_audio_dir, audio_filename)
            sf.write(audio_path, audio_array, sample_rate)

            records.append({
                "audio_filename": audio_filename,
                "text": example[self.text_column],
                "duration": example["duration"],
                "duration_sec": duration_sec,  # No rounding
                "duration_ms": duration_ms    # No rounding
            })

        df = pd.DataFrame(records)
        return df

    def validate_audio_files(self, df):
        print("Validating existence of audio files...")
        available_files = set(os.listdir(self.output_audio_dir))
        df['found'] = df['audio_filename'].apply(lambda x: 'yes' if x in available_files else 'no')
        return df

    def remove_duplicates(self, df):
        print("Removing duplicates...")
        df_unique = df.drop_duplicates(subset=["audio_filename"])
        return df_unique

    def rename_and_reorder_columns(self, df):
        print("Renaming and reordering columns...")
        df.rename(columns={
            "audio_filename": "path",
            "text": "sentence"
        }, inplace=True)

        # Move 'found' column after 'sentence'
        cols = list(df.columns)
        found_idx = cols.index("found")
        cols.insert(cols.index("sentence") + 1, cols.pop(found_idx))  # move 'found' after 'sentence'
        df = df[cols]
        return df

    def remove_large_duration_rows(self, df):
        print("Removing rows with duration_sec > 15...")
        df_filtered = df[df["duration_sec"] <= 15]
        return df_filtered

    def check_and_copy_audio_files(self, df):
        print("Checking and copying audio files to final folder...")
        for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
            audio_filename = row["path"]
            audio_path = os.path.join(self.output_audio_dir, audio_filename)
            if os.path.exists(audio_path):
                shutil.copy(audio_path, self.final_audio_folder)

    def drop_duration_column_if_same(self, df):
        if df["duration"].equals(df["duration_sec"]):
            print("Dropping 'duration' column as it's the same as 'duration_sec'...")
            df.drop(columns=["duration"], inplace=True)
        return df

    def generate_summary(self, df):
        print("\n DataFrame Summary:")
        print(df.info())
        print("\n Describe:")
        print(df.describe())
        print("\n Head:")
        print(df.head())
        print("\n Tail:")
        print(df.tail())
        print("\n Unique Audio Files:", df['path'].nunique())

        # Calculate the total durations
        total_duration = df['duration'].sum()
        total_duration_sec = df['duration_sec'].sum()
        total_duration_ms = df['duration_ms'].sum()
        print(f"\nTotal Duration (sec): {total_duration}")
        print(f"Total Duration (ms): {total_duration_ms}")
        print(f"Total Duration (seconds): {total_duration_sec}")

    def run_pipeline(self):
        df = self.extract_and_save_audio()
        df = self.validate_audio_files(df)
        df = self.remove_duplicates(df)
        df = self.rename_and_reorder_columns(df)
        df = self.remove_large_duration_rows(df)

        # Save filtered CSV
        df.to_csv(self.output_csv_path, index=False)

        # Create final folder and copy audio files
        self.check_and_copy_audio_files(df)

        # Drop duration column if 'duration' and 'duration_sec' are the same
        df = self.drop_duration_column_if_same(df)

        # Save the filtered file
        df.to_csv("/mnt/data/stt/Datasets/Datasets/filtered_gujarati_BA_spo-tut.csv", index=False)

        # Generate summary and statistics
        self.generate_summary(df)

        print(f"\n Final CSV saved at: {self.output_csv_path}")


# === Run the processor in Jupyter ===
processor = IndicVoiceProcessor(
    dataset_name="ai4bharat/Spoken-Tutorial",
    language="gujarati",
    audio_column="chunked_audio_filepath",
    text_column="text",
    output_audio_dir="/mnt/data/stt/Datasets/Datasets/gujarati_audio_chunks_ba_spo-tut",
    output_csv_path="/mnt/data/stt/Datasets/Datasets/filtered_gujarati_dataset_ba_spo-tut.csv",
    final_audio_folder="/mnt/data/stt/Datasets/Datasets/gujarati_audios_BA_spo-tut_final"
)

processor.run_pipeline()


Extracting and saving audio files...


100%|██████████| 34647/34647 [02:35<00:00, 222.95it/s]


Validating existence of audio files...
Removing duplicates...
Renaming and reordering columns...
Removing rows with duration_sec > 15...
Checking and copying audio files to final folder...


100%|██████████| 33430/33430 [00:52<00:00, 635.36it/s]



 DataFrame Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 33430 entries, 0 to 34646
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   path          33430 non-null  object 
 1   sentence      33430 non-null  object 
 2   found         33430 non-null  object 
 3   duration      33430 non-null  float64
 4   duration_sec  33430 non-null  float64
 5   duration_ms   33430 non-null  float64
dtypes: float64(3), object(3)
memory usage: 1.8+ MB
None

 Describe:
           duration  duration_sec   duration_ms
count  33430.000000  33430.000000  33430.000000
mean       4.891269      4.890914   4890.913719
std        2.810614      2.810900   2810.900272
min        0.080000      0.080000     80.000000
25%        2.880000      2.880000   2880.000000
50%        4.320000      4.320000   4320.000000
75%        6.240000      6.240000   6240.000000
max       14.960000     14.960000  14960.000000

 Head:
                       

**Dropping duration column**

In [12]:
import pandas as pd
csv_path = "/mnt/data/stt/Datasets/Datasets/filtered_gujarati_BA_spo-tut.csv" # Load the CSV file
df = pd.read_csv(csv_path)


print(f" Total columns before dropping: {df.shape[1]}") # Show total number of columns before dropping
print(" Column names:", df.columns.tolist())


if 'duration' in df.columns: # Drop the 'duration' column if it exists
    df.drop(columns=['duration'], inplace=True)
    print(" 'duration' column dropped.")
else:
    print(" 'duration' column not found.")


print(f" Total columns after dropping: {df.shape[1]}") # Show total columns after dropping
print(" Updated column names:", df.columns.tolist())


 Total columns before dropping: 6
 Column names: ['path', 'sentence', 'found', 'duration', 'duration_sec', 'duration_ms']
 'duration' column dropped.
 Total columns after dropping: 5
 Updated column names: ['path', 'sentence', 'found', 'duration_sec', 'duration_ms']


In [13]:
df.to_csv(csv_path, index=False)
print(" Updated CSV saved.")

 Updated CSV saved.


**Playing updated audios**

In [14]:
audio_folder = "/mnt/data/stt/Datasets/Datasets/gujarati_audios_BA_spo-tut_final"
# List all .wav files
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
# Count them
print(f" Total number of audio files: {len(audio_files)}")

 Total number of audio files: 33430


In [15]:
audio_files = sorted(os.listdir(audio_folder))[:5]  # first 5 files

for file in audio_files:
    print(f" Playing: {file}")
    display(Audio(filename=os.path.join(audio_folder, file)))

 Playing: bashaanuvaadh_spo-tut_gu_0.wav


 Playing: bashaanuvaadh_spo-tut_gu_10.wav


 Playing: bashaanuvaadh_spo-tut_gu_100.wav


 Playing: bashaanuvaadh_spo-tut_gu_1000.wav


 Playing: bashaanuvaadh_spo-tut_gu_10000.wav
